In [ ]:
import pandas as pd
from tabulate import tabulate

In [ ]:
df1 = pd.read_csv('cleaned_stock_data.csv')
df2 = pd.read_csv('world_bank_macro_data_standardized_sheet-xls - all_data.csv')

In [ ]:
print(df1.head())
print(df2.head())

In [ ]:
print(df1.info())
print("\n")
print(df2.info())

In [ ]:
print(df1)
print(df2)

In [ ]:
japan_df = df2[df2['Country'] == 'Japan']
print(japan_df)

In [ ]:
index_country_map = {
    "S&P 500 (USA)" : "USA",
    "WTI Crude Oil (CL=F)" : "GLOBAL",
    "DAX (Germany)" : "DEU",
    "Nifty 50 (India)" : "IND",
    "Shanghai Composite (China)": "CHN" ,
    "Nikkei 225 (Japan)" : "JPN"
}

In [ ]:
print(index_country_map.values())

In [ ]:
df1['Date'] = pd.to_datetime(df1['Date'])
df1['Year'] = df1['Date'].dt.year

In [ ]:
grouped = df1.sort_values('Date').groupby(['Index', 'Year'])
annual = grouped.agg(First_Close=('Close','first'), Last_Close=('Close','last')).reset_index()
print(annual)

In [ ]:
# annual.drop('annual_return_pct', axis=1, inplace=True)

In [ ]:
annual['Annual_Return_Pct'] = (((annual['Last_Close']) - (annual['First_Close'])) / (annual['First_Close'])) * 100
annual['Annual_Return_Pct'] = annual['Annual_Return_Pct'].round()
print(annual[['Index', 'Year', 'Annual_Return_Pct']])

In [ ]:
print(df1.head())

In [ ]:
df1['CountryCode'] = df1['Index'].map(index_country_map)

In [ ]:
print(df1[['Index','CountryCode']])

In [ ]:
oil_df = df1[df1['CountryCode'] == 'GLOBAL'].copy()

In [ ]:
print(oil_df)

In [ ]:
annual['CountryCode'] = annual['Index'].map(index_country_map)

In [ ]:
print(annual)

In [ ]:
print(df2)

In [ ]:
annual_no_oil = annual[annual['CountryCode'] != 'GLOBAL']
merged_df = pd.merge(
    annual_no_oil,
    df2,
    on=['CountryCode','Year'],
    how='inner'
)

In [ ]:
print(merged_df)

In [ ]:
correlation = merged_df.corr(numeric_only=True)['Annual_Return_Pct'].sort_values(ascending=False)
print(correlation)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.scatterplot(data=merged_df, x='Inflation (Annual %)', y='Annual_Return_Pct', hue='Country')
plt.title("Stock Return vs Inflation")
plt.grid(True)
plt.show()

In [ ]:
merged_df.groupby('Year')['Annual_Return_Pct'].mean().plot(marker='o', title='Average Stock Returns Over Years')
plt.ylabel("Avg Annual Return (%)")
plt.grid(True)
plt.show()

In [ ]:
top_growth = merged_df[merged_df['GDP Growth (Annual %)'] > 5]
top = top_growth.sort_values(by='Annual_Return_Pct', ascending=False)[['Year', 'Country', 'Annual_Return_Pct', 'GDP Growth (Annual %)']]
print(top.head(10))

In [ ]:
# Can we guess the stock return by looking at economic indicators?

from sklearn.linear_model import LinearRegression

# X is a table of 4 macro indicators (our inputs/features)
X = merged_df[['Inflation (Annual %)', 'GDP Growth (Annual %)', 'FDI Net Inflows (% of GDP)', 'Gross Savings (% of GDP)']]
# y is the actual annual stock return (our output/target)
y = merged_df['Annual_Return_Pct']

model = LinearRegression()
model.fit(X, y)

print("Coefficients:", dict(zip(X.columns, model.coef_)))
print("Intercept:", model.intercept_)

In [ ]:
# R² Score (Goodness of Fit)
from sklearn.metrics import r2_score

# Predict values
y_pred = model.predict(X)

# Calculate R²
r2 = r2_score(y, y_pred)
print(f"R² score: {r2:.4f}")

In [ ]:
# p-values (statistical significance)
import statsmodels.api as sm

# Add constant for intercept
X_sm = sm.add_constant(X)

# Fit OLS model
ols_model = sm.OLS(y, X_sm).fit()

# Summary includes p-values, R², etc.
print(ols_model.summary())

# Model Fit: Poor (R² = 0.16) — the macro variables barely explain the variation in stock returns.
# Predictors: None of the independent variables are statistically significant at 95% confidence, though Gross Savings comes close.

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(y, y_pred, color='blue', label='Predictions')
plt.plot([-20, 30], [-20, 30], color='red', linewidth=2, label='Ideal Fit Line')
plt.xlabel("Actual Annual Return (%)")
plt.ylabel("Predicted Annual Return (%)")
plt.title("Actual vs. Predicted Returns")
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
# Only numeric columns of interest
corr_columns = ['Annual_Return_Pct', 'Inflation (Annual %)', 'GDP Growth (Annual %)',
                'FDI Net Inflows (% of GDP)', 'Gross Savings (% of GDP)']
corr = merged_df[corr_columns].corr()

plt.figure(figsize=(8, 6))
sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Between Macro Indicators and Annual Stock Returns")
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.lineplot(data=merged_df, x='Year', y='Annual_Return_Pct', hue='Country', marker='o')
plt.title("Annual Stock Returns Over Time by Country")
plt.ylabel("Annual Return (%)")
plt.xlabel("Year")
plt.legend(title='Country')
plt.grid(True)
plt.show()

In [ ]:
# Filter for India only
india_df = merged_df[merged_df['Country'] == 'India']

# Choose relevant numeric columns
corr_columns = [
    'Annual_Return_Pct',
    'Inflation (Annual %)',
    'GDP Growth (Annual %)',
    'FDI Net Inflows (% of GDP)',
    'Gross Savings (% of GDP)'
]

# Compute correlation matrix
india_corr = india_df[corr_columns].corr()

# Plot heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(india_corr, annot=True, cmap="YlGnBu", fmt=".2f")
plt.title("Correlation Heatmap: India Only")
plt.show()

In [ ]:
# Filter for USA only
usa_df = merged_df[merged_df['Country'] == 'USA']

# Select relevant numeric columns
corr_columns = [
    'Annual_Return_Pct',
    'Inflation (Annual %)',
    'GDP Growth (Annual %)',
    'FDI Net Inflows (% of GDP)',
    'Gross Savings (% of GDP)'
]

# Compute correlation matrix
usa_corr = usa_df[corr_columns].corr()

# Plot heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(usa_corr, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Heatmap: USA Only")
plt.show()

# A positive correlation (close to +1) suggests both rise together.
# A negative correlation (close to -1) suggests one rises while the other falls.
# Near 0 means little to no relationship.